In [41]:
import pandas as pd
import re

# Dataset info

In [42]:
df = pd.read_csv('../data/raw/NLP training data 0816.csv')
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, )
df.head()

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description
0,0,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,Which plan(s) cover preventive benefits?
1,1,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,How much does preventive benefits cost to me?
2,2,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,Which plan(s) cover preventive benefits?
3,3,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,How much does preventive benefits cost to me?
4,4,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Alcohol misuse screenings & counseling,Which plan(s) cover preventive benefits?


# Generation

## which plans

In [43]:
df.shape, df.question_description.unique().shape

((11434, 8), (284,))

In [44]:
mask = df.question_description.str.contains(']', regex=False)
df[mask]

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description


In [45]:
mask = df.question_description.str.contains('Which plan', regex=False)
df[mask].shape, df[mask].sort_values(by='question_description').question_description.unique().shape

((8409, 8), (107,))

In [46]:
df[mask].question_description.unique()

array(['Which plan(s) cover preventive benefits?',
       'Which plan(s) cover outpatient hospital services?',
       'Which plan(s) cover ambulatory surgical center (ASC)?',
       'Which plan(s) cover outpatient hospital observation services?',
       'Which plan(s) cover outpatient breast procedure?',
       'Which plan(s) cover outpatient cardiovascular procedure?',
       'Which plan(s) cover outpatient digestive/gastrointestinal procedure?',
       'Which plan(s) cover outpatient eye procedure?',
       'Which plan(s) cover outpatient hematology procedure?',
       'Which plan(s) cover outpatient musculoskeletal procedure?',
       'Which plan(s) cover outpatient skin procedure?',
       'Which plan(s) cover outpatient vascular procedure?',
       'Which plan(s) cover outpatient dialysis procedure?',
       'Which plan(s) cover chemotherapy drugs?',
       'Which plan(s) cover durable medical equipment (e.g. wheelchair, oxygen)?',
       'Which plan(s) cover blood sugar meters?',

In [47]:
processed_df = df.copy()

def which_plan(line):
    
    
    if 'and what\'s the cost' in line['question_description']:
        line['question_description'] = re.sub(r"(?P<q>Which plan\(s\) cover) (?P<entity>.[^?]+)(?P<e> and what's the cost\?)", r'\g<q> [\g<entity>]\g<e>', line['question_description'])
        line['question_type'] = 0
    
    elif 'Which plan' in line['question_description']:
        if 'coverage?' not in line['question_description']:
            line['question_description'] = re.sub(r'(?P<q>cover|offer) (?P<entity>.[^?]+)', r'\g<q> [\g<entity>]', line['question_description'])  
        else:
            line['question_description'] = re.sub(r'(?P<q>cover|offer|have) (?P<entity>.[^?]+)(?P<e> coverage\?)', r'\g<q> [\g<entity>]\g<e>', line['question_description'])  
        line['question_type'] = 1
    
    elif 'How much does ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(How much does)) (?P<entity>.[^?]+)(?P<e> cost to me\?)', r'\g<q> [\g<entity>]\g<e>', line['question_description'])
        line['question_type'] = 2
    
    elif 'What does ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(What does)) (?P<entity>.[^?]+)(?P<e> mean\?)', r'\g<q> [\g<entity>]\g<e>', line['question_description'])
        line['question_type'] = 3
    
    elif 'What is the ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(What is the)) (?P<entity>.[^?]+)', r'\g<q> [\g<entity>]', line['question_description'])
        line['question_type'] = 3
        
    elif 'I\'d like to talk to ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(I\'d like to talk to)) (?P<entity>.[^?]+)', r'\g<q> [\g<entity>]', line['question_description'])
        line['question_type'] = 4
    
    return line
processed_df = processed_df.apply(which_plan, axis='columns')

In [48]:
mask = processed_df.question_description.str.contains('Which plan', regex=False)
processed_df[mask].question_description.unique()

array(['Which plan(s) cover [preventive benefits]?',
       'Which plan(s) cover [outpatient hospital services]?',
       'Which plan(s) cover [ambulatory surgical center (ASC)]?',
       'Which plan(s) cover [outpatient hospital observation services]?',
       'Which plan(s) cover [outpatient breast procedure]?',
       'Which plan(s) cover [outpatient cardiovascular procedure]?',
       'Which plan(s) cover [outpatient digestive/gastrointestinal procedure]?',
       'Which plan(s) cover [outpatient eye procedure]?',
       'Which plan(s) cover [outpatient hematology procedure]?',
       'Which plan(s) cover [outpatient musculoskeletal procedure]?',
       'Which plan(s) cover [outpatient skin procedure]?',
       'Which plan(s) cover [outpatient vascular procedure]?',
       'Which plan(s) cover [outpatient dialysis procedure]?',
       'Which plan(s) cover [chemotherapy drugs]?',
       'Which plan(s) cover [durable medical equipment (e.g. wheelchair, oxygen)]?',
       'Which plan(

In [49]:
mask = processed_df.question_description.str.contains('How much does ', regex=False)
processed_df[mask].question_description.unique()

array(['How much does [preventive benefits] cost to me?',
       'How much does [outpatient hospital services] cost to me?',
       'How much does [ambulatory surgical center (ASC)] cost to me?',
       'How much does [outpatient hospital observation services] cost to me?',
       'How much does [outpatient breast procedure] cost to me?',
       'How much does [outpatient cardiovascular procedure] cost to me?',
       'How much does [outpatient digestive/gastrointestinal procedure] cost to me?',
       'How much does [outpatient eye procedure] cost to me?',
       'How much does [outpatient hematology procedure] cost to me?',
       'How much does [outpatient musculoskeletal procedure] cost to me?',
       'How much does [outpatient skin procedure] cost to me?',
       'How much does [outpatient vascular procedure] cost to me?',
       'How much does [outpatient dialysis procedure] cost to me?',
       'How much does [chemotherapy drugs] cost to me?',
       'How much does [durable med

In [50]:
mask = ~processed_df.question_description.str.contains(']', regex=False)
processed_df[mask].question_description.unique()

array([], dtype=object)

# Merge keywords and phrases

In [51]:
processed_df.head()

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description,question_type
0,0,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,Which plan(s) cover [preventive benefits]?,1
1,1,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,How much does [preventive benefits] cost to me?,2
2,2,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,Which plan(s) cover [preventive benefits]?,1
3,3,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,How much does [preventive benefits] cost to me?,2
4,4,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Alcohol misuse screenings & counseling,Which plan(s) cover [preventive benefits]?,1


In [52]:
merged_df = processed_df.copy()
merged_df.drop(['campaign_id', 'client_id', 'index', 'question_category'], axis='columns', inplace=True)

def uncapitalize(s):
    return s[:1].lower() + s[1:] if s else ''
    
def merge_data(line):
    if isinstance(line['question_keyword'], str):
        line['train_phase'] = re.sub(r'\[[^?]+\]', uncapitalize(line['question_keyword']), line['question_description'])
    else:
        line['train_phase'] = line['question_description']
    return line
merged_df = merged_df.apply(merge_data, axis='columns')
merged_df.head()

,question_id,question_subcategory,question_keyword,question_description,question_type,train_phase
0,benefit_search-preventive_care-1,preventive care,Preventive care,Which plan(s) cover [preventive benefits]?,1,Which plan(s) cover preventive care?
1,benefit_search-preventive_care-2,preventive care,Preventive care,How much does [preventive benefits] cost to me?,2,How much does preventive care cost to me?
2,benefit_search-preventive_care-1,preventive care,Abdominal aortic aneurysm screening,Which plan(s) cover [preventive benefits]?,1,Which plan(s) cover abdominal aortic aneurysm ...
3,benefit_search-preventive_care-2,preventive care,Abdominal aortic aneurysm screening,How much does [preventive benefits] cost to me?,2,How much does abdominal aortic aneurysm screen...
4,benefit_search-preventive_care-1,preventive care,Alcohol misuse screenings & counseling,Which plan(s) cover [preventive benefits]?,1,Which plan(s) cover alcohol misuse screenings ...


In [53]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11434 entries, 0 to 11433
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   question_id           11434 non-null  object
 1   question_subcategory  11359 non-null  object
 2   question_keyword      11433 non-null  object
 3   question_description  11434 non-null  object
 4   question_type         11434 non-null  int64 
 5   train_phase           11434 non-null  object
dtypes: int64(1), object(5)
memory usage: 536.1+ KB


In [54]:
merged_df[merged_df.train_phase.str.contains('\[')]

,question_id,question_subcategory,question_keyword,question_description,question_type,train_phase
288,benefit_search-outpatient_musculoskeletal_proc...,outpatient musculoskeletal procedure,"Open treatment of ulnar fracture, proximal end...",Which plan(s) cover [outpatient musculoskeleta...,1,Which plan(s) cover open treatment of ulnar fr...
289,benefit_search-outpatient_musculoskeletal_proc...,outpatient musculoskeletal procedure,"Open treatment of ulnar fracture, proximal end...",How much does [outpatient musculoskeletal proc...,2,How much does open treatment of ulnar fracture...
820,benefit_search-outpatient_musculoskeletal_proc...,outpatient musculoskeletal procedure,"Laminectomy, facetectomy and foraminotomy (uni...",Which plan(s) cover [outpatient musculoskeleta...,1,"Which plan(s) cover laminectomy, facetectomy a..."
821,benefit_search-outpatient_musculoskeletal_proc...,outpatient musculoskeletal procedure,"Laminectomy, facetectomy and foraminotomy (uni...",How much does [outpatient musculoskeletal proc...,2,"How much does laminectomy, facetectomy and for..."
822,benefit_search-outpatient_musculoskeletal_proc...,outpatient musculoskeletal procedure,"Laminectomy, facetectomy and foraminotomy (uni...",Which plan(s) cover [outpatient musculoskeleta...,1,"Which plan(s) cover laminectomy, facetectomy a..."
...,...,...,...,...,...,...
4829,benefit_search-lab_services-1,lab services,Flow cytometry detection of aquaporin-4 (neuro...,Which plan(s) cover [lab services]?,1,Which plan(s) cover flow cytometry detection o...
4830,benefit_search-lab_services-2,lab services,Flow cytometry detection of aquaporin-4 (neuro...,How much does [lab services] cost to me?,2,How much does flow cytometry detection of aqua...
6684,benefit_search-pharmacy-1,pharmacy,NaN,Which plan(s) cover [my drug(s)] and what's th...,0,Which plan(s) cover [my drug(s)] and what's th...
9351,benefit_search-pharmacy-1,pharmacy,HUMAN-BOVINE REASSORTANT ROTAVIRUS STRAIN G1 V...,Which plan(s) cover [my drug(s)] and what's th...,0,Which plan(s) cover hUMAN-BOVINE REASSORTANT R...


In [55]:
merged_df.question_type.value_counts()

0    5473
2    2941
1    2936
3      80
4       4
Name: question_type, dtype: int64

In [56]:
merged_df.to_csv('../data/interim/processed_dataset.csv', index=False)

# Live agent

In [46]:
df = pd.read_csv('../data/external/rasa_ask_human.csv')
df.head()

,phrase
0,Can I speak to anyone who can really help me?
1,Could I talk to [Tyrone King](name)?
2,I don't wanna talk to a bot
3,I dont like to talk to a machine
4,I want to talk to a human


In [ ]:
df.drop_duplicates(inplace=True)
df.to_csv('../data/external/rasa_ask_human_unique.csv', index=False)